In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [148]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    # at = Nt/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    # at = Nt/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    # ua,svam,vha=la.svd(Am)
    # va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

# without symmetry
## loading data

In [240]:
'''
without symmetry eigenvalue and eigenvector projection 
'''
# strfile='2021-03-27-16_16_38_perturbationHomoZeroeta_data' # 500
strfile='2021-03-27-17_02_42_perturbationHomoZeroeta_data' # 1000
# strfile='2021-03-27-17_58_42_perturbationHomoZeroeta_data' # 750
# strfile='2021-03-27-20_31_10_perturbationHomoZeroeta_data' # 750
# strfile='2021-03-27-20_44_33_perturbationHomoZeroeta_data' # 750

# strfile='2021-03-27-20_01_56_perturbationHeterZeroeta_data' # 750
'''
previous data
'''
# strfile='2021-03-17-21_57_03_perturbationHomoZeroeta_data' # N=500
# strfile='2021-03-17-22_24_20_perturbationHomoZeroeta_data' #1000
# strfile='2021-03-17-20_35_21_perturbationHomoZeroeta_data' # 1000
# strfile='2021-03-17-22_46_39_perturbationHomoZeroeta_data'# 750
# '''# HETERO ZERO '''
# strfile='2021-03-13-15_10_37_perturbationHeterZeroeta_data' 
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
ngavg=len(gaverageseries)
eigvAm=data['eigvAm']
eigvalseries=data['eigvalseries']
statesReig=data['statesReig']
statesLeig=data['statesLeig']
meigvecAm=data['meigvecAm']
neigvecAm=data['neigvecAm']
Xprimebasexee=data['Xprimebasexee']
Jparameters=data['Jparameters']
# right and left eigenvectors
Reigvecseries,Leigvecseries,Beigvseries=data['Reigvecseries'],data['Leigvecseries'],data['Beigvseries']
ntrial=np.shape(Reigvecseries)[1]
x0,y0=data['x0'],data['y0']
Randgain=data['Randgain']
xee,xei,xie,xii=Randgain[0],Randgain[1],Randgain[2],Randgain[3]

print('scale:',Jparameters[4])
NE=int(Jparameters[4])
print('JEE:',Jparameters[0])
etaset=data['etaset']
signeta=data['signeta']
x0,y0=data['x0'],data['y0']
JE,JI=Jparameters[0],Jparameters[1]
print('eta:',etaset,'; signs:',signeta)

scale: 1000.0
JEE: 1.0
eta: [0. 0. 0.] ; signs: [1. 1. 1.]


In [241]:
'''
>>>>>>>>>>>>>>>>> Compare the individual response (not statistics), NOTE:
To make direct comparison between numerical and perturbation theorem, keep the same random matrix for each trial
choose one trial -- determine the random derivative -- calculate the individual elements in vector

'''

'''
# loading data
# strfile='2021-03-13-15_20_04_perturbationHomoZeroeta_data'# homogeneous random gain
strfile='2021-03-13-15_10_37_perturbationHeterZeroeta_data' # heterogeneous random gain
data = np.load(strfile+'.npz')
gaverageseries = data['gaverageseries']
x0,y0=data['x0'],data['y0']
meigvecAm,neigvecAm=data['meigvecAm'],data['neigvecAm']
# perturbed
Leigvecseries,Reigvecseries=data['Leigvecseries'],data['Reigvecseries']
Xprimebasexee=data['Xprimebasexee']
eigvAm=data['eigvAm']
ngavg,ntrial,N=np.shape(Leigvecseries)
Leigvecseries=np.expand_dims(Leigvecseries,axis=3)
Reigvecseries=np.expand_dims(Reigvecseries,axis=3)
NE=int(N/2.0)
'''

# '''
# calculate S0
# '''
# xticks = np.linspace(-0.08,0.08,2)
# xlims = [-0.08,0.08]
# yticks = np.linspace(-0.08,0.08,2)
# ylims = [-0.08,0.08]
# Z = np.zeros((NE*2-1,NE*2-1))
# for i in range(NE*2-1):
#     b1=eigvAm[i+1]
#     Z[i,i]=1/(-eigvAm[0])
# S0=np.zeros((NE*2,NE*2))
# R1,L1=np.squeeze(meigvecAm[:,1:]),np.squeeze(neigvecAm[:,1:])
# S0=R1@Z@L1.T

'''
# CHOOSE ONE TRIAL
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])

for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])
'''
# CHOOSE TWO TRIALS
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial_,:])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial_,:])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])



for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])

<ipython-input-241-cdfb9a45373f>:53: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(y0)
<ipython-input-241-cdfb9a45373f>:54: ComplexWarning: Casting complex values to real discards the imaginary part
  rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(x0)
<ipython-input-241-cdfb9a45373f>:56: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_the[:,i] = lprime_the*gsamples[i]
<ipython-input-241-cdfb9a45373f>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  rperturb_the[:,i] = rprime_the*gsamples[i]
<ipython-input-241-cdfb9a45373f>:53: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(y0)
<ipython-input-241-cdfb9a45373f>:54: ComplexWarning: Casting complex values to r

In [242]:
'''
### >>>>>>>>>>>>>>>>> Statistics Linearly Change >>>>>>>>>>>>>>>>
Case 1,2 focus on second-order statstics WITHIN (left/right) eigenvectors
Calculate the statistics of eigenvalue, eigenvectors
  * mean, variance of the perturbation of eigenvalue \mu(derivative of lambda),\sigma(derivative of lambda)
'''
'''
Variables:
    std_perturb_Lambda_num (1)
    std_perturb_Lambda_theo (1)
    std_perturbLambda_linear: reconstruct np.zeros(ngavg)
    std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
'''
alphanum=1.0
# figE,axE=plt.subplots(figsize=(4,4))
# figL0,axL0=plt.subplots(2,1,figsize=(5,4))
# figR0,axR0=plt.subplots(2,1,figsize=(5,4))

## calculating the statistics of perturbation lambda
# Coding uses only one point [linear assumption]
std_perturb_Lambda_numSE  = np.std(eigvalseries[ngavg-1,:,0])
std_perturb_Lambda_theo = np.sqrt((JE**2*(xee**2+xei**2)+JI**2*(xie**2+xii**2))/(2*NE)/(JE-JI)**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
std_perturbLambda_linear_numSE=std_perturb_Lambda_numSE*gaverageseries
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Lambda_num = np.std(Beigvseries[:,:],axis=1)
axE.plot(gaverageseries,std_perturb_Lambda_num,color='darkgreen',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
# polyfitting
p=np.polyfit(gaverageseries,std_perturb_Lambda_num,1)
std_perturbLambda_linear_num = np.zeros(ngavg)
std_perturbLambda_linear_num = p[0]*gaverageseries+p[1]
# axE.plot(gaverageseries,std_perturbLambda_linear,color=cm[2],linestyle='--',linewidth=1.5,alpha=0.5,label='1-order polynomial')
axE.plot(gaverageseries,std_perturbLambda_linear_theo,color='green',linewidth=1.5,label='analytical',alpha=alphanum)
print('std delta_lambda numerical:',p[0])
axE.set_title(r'$\sigma_{\lambda}$')

'''
plt and ax
'''
ylims=[-0.00,0.08]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.08,2)
xticks = np.linspace(0.0,1.0,2)
axE.set_xlim(xlims)
axE.set_xticks(xticks)
axE.set_ylim(ylims)
axE.set_yticks(yticks)
# ax.legend()

'''
right eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Reig_numSE,std_perturb_Reig_theo=np.zeros(2),np.zeros(2)
std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbReig_linear_theo,std_perturbReig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbReig_linear_theo[:,0]=std_perturb_Reig_theo[0]*gaverageseries
std_perturbReig_linear_theo[:,1]=std_perturb_Reig_theo[1]*gaverageseries
# std_perturbReig_linear_numSE[:,0]=std_perturb_Reig_numSE[0]*gaverageseries
# std_perturbReig_linear_numSE[:,1]=std_perturb_Reig_numSE[1]*gaverageseries

print('std of perturbation right eigenvector, num:',std_perturb_Reig_numSE,'; theo:',std_perturb_Reig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Reig_num,std_perturbReig_linear_num=np.zeros((ngavg,2),dtype=complex),np.zeros((ngavg,2),dtype=complex)
std_perturb_Reig_num[:,0],std_perturb_Reig_num[:,1] = np.mean(statesReig[:,:,0,1],axis=1),np.mean(statesReig[:,:,1,1],axis=1)


# >>>>>>>> numerical solution
axR0[0].plot(gaverageseries,std_perturb_Reig_num[:,0],color='darkred',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
axR0[1].plot(gaverageseries,std_perturb_Reig_num[:,1],color='darkblue',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
# axR0[0].scatter(gaverageseries,std_perturbReig_linear_numSE[:,0],color='orange',label='numerical')
# axR0[1].scatter(gaverageseries,std_perturbReig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axR0[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical',alpha=alphanum)
axR0[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical',alpha=alphanum)
axR0[0].set_title(r'$\sigma_{r}^E$')
axR0[1].set_title(r'$\sigma_{r}^I$')
# >>>>>>>> numerical solution
for i in range(ntrial):
    axR0[0].scatter(gaverageseries,statesReig[:,i,0,1],s=5,c='darkred',marker='^',label='numerical',alpha=alphanum*0.5)
    axR0[1].scatter(gaverageseries,statesReig[:,i,1,1],s=5,c='darkblue',marker='^',label='numerical',alpha=alphanum*0.5)

ylims=[-0.00,0.020]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.020,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axR0[i].set_xlim(xlims)
    axR0[i].set_xticks([])
    axR0[i].set_ylim(ylims)
    axR0[i].set_yticks(yticks)
axR0[1].set_xticks(xticks)

'''
left eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Leig_numSE,std_perturb_Leig_theo=np.zeros(2),np.zeros(2)
std_perturb_Leig_numSE[0],std_perturb_Leig_numSE[1] = np.mean(statesLeig[ngavg-1,:,0,1]),np.mean(statesLeig[ngavg-1,:,1,1])
std_perturb_Leig_theo[0],std_perturb_Leig_theo[1] = np.sqrt(2*((JE*xee)**2+(JI*xie)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2),np.sqrt(2*((JE*xei)**2+(JI*xii)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLeig_linear_theo,std_perturbLeig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbLeig_linear_theo[:,0]=std_perturb_Leig_theo[0]*gaverageseries
std_perturbLeig_linear_theo[:,1]=std_perturb_Leig_theo[1]*gaverageseries
std_perturbLeig_linear_numSE[:,0]=std_perturb_Leig_numSE[0]*gaverageseries
std_perturbLeig_linear_numSE[:,1]=std_perturb_Leig_numSE[1]*gaverageseries

print('std of perturbation left eigenvector, num:',std_perturb_Leig_numSE,'; theo:',std_perturb_Leig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Leig_num,std_perturbLeig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Leig_num[:,0],std_perturb_Leig_num[:,1] = np.mean(statesLeig[:,:,0,1],axis=1),np.mean(statesLeig[:,:,1,1],axis=1)

# >>>>>>>> numerical solution
axL0[0].plot(gaverageseries,std_perturb_Leig_num[:,0],color='darkred',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
axL0[1].plot(gaverageseries,std_perturb_Leig_num[:,1],color='darkblue',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
# axL0[0].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,0],color='orange',label='numerical')
# axL0[1].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axL0[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical',alpha=alphanum)
axL0[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical',alpha=alphanum)
axL0[0].set_title(r'$\sigma_{l}^E$')
axL0[1].set_title(r'$\sigma_{l}^I$')
axL0[1].set_xticks(xticks)
# >>>>>>>> numerical solution
for i in range(ntrial):
    axL0[0].scatter(gaverageseries,statesLeig[:,i,0,1],s=5,c='darkred',marker='^',label='numerical',alpha=alphanum*0.5)
    axL0[1].scatter(gaverageseries,statesLeig[:,i,1,1],s=5,c='darkblue',marker='^',label='numerical',alpha=alphanum*0.5)

ylims=[-0.00,0.045]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.045,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axL0[i].set_xlim(xlims)
    axL0[i].set_xticks([])
    axL0[i].set_ylim(ylims)
    axL0[i].set_yticks(yticks)
axL0[1].set_xticks(xticks)





std delta_lambda numerical: 0.041163186069500146
std of perturbation right eigenvector, num: [0.00966569 0.00973817] ; theo: [0.00894427 0.00894427]
std of perturbation left eigenvector, num: [0.01994889 0.01988402] ; theo: [0.01841738 0.01841738]
<ipython-input-242-169649c8e34f>:60: ComplexWarning: Casting complex values to real discards the imaginary part
  std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
<ipython-input-242-169649c8e34f>:61: ComplexWarning: Casting complex values to real discards the imaginary part
  std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/

In [58]:
from scipy.special import comb, perm
C=comb(4,2)
print(C)

6.0


## eigenvalue spectrum

In [ ]:
'''
first, generate eigenvalues' spectrum (we didn't record the whole eigenvalues' spectrum)
'''

In [206]:
'''
without symmetry, 
eigvAm
'''
idxgavg,idxtrial=0,8
figtspt,axtspt=plt.subplots(figsize=(8,5))
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1:]),np.imag(Beigvseries[idxgavg,idxtrial,1:]),s=10,c='blue',alpha=0.5)
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0]),s=20,c='red',marker='^',alpha=0.5)
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
#设置x坐标刻度数字或名称的位置
axtspt.xaxis.set_ticks_position('bottom')
#设置边框位置
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
xticks = np.linspace(-1.3,1.3,2)
xlims = [-1.3,1.3]
yticks = np.linspace(-1.3,1.3,2)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverageseries[idxgavg]
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
eta=0
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="darkblue", linewidth=2.5,linestyle='--',label=r'ellipse')
avgg=np.sum(np.sum(gmat))*2/4.0
avgg=np.sqrt(avgg)
xa = avgg*np.cos(theta)
ya = avgg*np.sin(theta)
axtspt.plot(xa, ya, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse')
print(avgg,gaverage_)



1.0 1.0 1.0 1.0
0.1
0.1 0.1
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [243]:
'''
perturbation direction and amplitude
covariance - non-covariance
'''
'''
# <l',r'> reflects the direction of changing
'''

'''
variance of r and variance of l <r.Tr>, <l.T,l>
overlap between r and l <l.Tr>
'''
xticks = np.linspace(-0.01,0.1,2)
xlims = [-0.01,0.1]
yticks = np.linspace(-0.01,0.1,2)
ylims = [-0.01,0.1]
## reconstruct Am
JEE,JEI,JIE,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
NI=NE
Nparams=np.array([NE,NI])
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
'''
for reference
'''
x0,y0=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))
gaverageseries=np.reshape(gaverageseries,(ngavg,1))
directionvecE,directionvecI,directionperturbE,directionperturbI=np.zeros((neta,ntrial,2,2)),np.zeros((neta,ntrial,2,2)),np.zeros((neta,ntrial,2,2)),np.zeros((neta,ntrial,2,2))
ampvecE,ampvecI,ampperturbE,ampperturbI=np.zeros((neta,ntrial,2)),np.zeros((neta,ntrial,2)),np.zeros((neta,ntrial,2)),np.zeros((neta,ntrial,2))
ntrial = np.shape(Leigvecseries)[1]
for itrial in range(ntrial):
    ## choose one trial and the random matrix
    # itrial=6
    derivJ=np.squeeze(Xprimebasexee[itrial,:,:])

    coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
    overlapperturb,overlapvector = np.zeros((neta,2,2,NE),dtype=complex),np.zeros((neta,2,2,NE),dtype=complex)
    # EI,LR,:
    Jprimemat=derivJ.copy()
    leftperturb=np.squeeze(-np.squeeze(y0.T@Jprimemat))/(-eigvAm[0])# left perturbation E
    rightperturb=np.squeeze(-np.squeeze(Jprimemat@x0))/(-eigvAm[0])#*Beigvseries[i,itrial,0]# right perturbation E

    # perturbEmat,perturbImat=np.zeros((2,NE)),np.zeros((2,NE))
    # perturbEmat[0,:],perturbEmat[1,:]=np.squeeze(leftperturb[:NE]),np.squeeze(rightperturb[:NE])
    # perturbImat[0,:],perturbImat[1,:]=np.squeeze(leftperturb[NE:]),np.squeeze(rightperturb[NE:])
    # pcovE,pcovI=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T
    # vpE,dirpE=la.eig(pcovE)
    # vpI,dirpI=la.eig(pcovI)
    # directionperturbE[i,itrial,:,:],directionperturbI[i,:,:]=dirpE,dirpI
    # ampperturbE[i,itrial,:],ampperturbI[i,itrial,:]=vpE*dh,vpI*dh

    overlapperturb[:,0,0,:],overlapperturb[:,0,1,:]=gaverageseries@np.reshape(leftperturb[:NE],(1,NE)),gaverageseries@np.reshape(rightperturb[:NE],(1,NE))
    overlapperturb[:,1,0,:],overlapperturb[:,1,1,:]=gaverageseries@np.reshape(leftperturb[NE:],(1,NE)),gaverageseries@np.reshape(rightperturb[NE:],(1,NE))
    meancol=np.reshape(np.mean(np.squeeze(Leigvecseries[:,itrial,:NE]),axis=1),(ngavg,1))
    meancol=np.repeat(meancol,NE,axis=1)
    overlapvector[:,0,0,:]=np.squeeze(Leigvecseries[:,itrial,:NE])-meancol
    meancol=np.reshape(np.mean(np.squeeze(Leigvecseries[:,itrial,NE:]),axis=1),(ngavg,1))
    meancol=np.repeat(meancol,NE,axis=1)
    overlapvector[:,1,0,:]=np.squeeze(Leigvecseries[:,itrial,NE:])-meancol
    meancol=np.reshape(np.mean(np.squeeze(Reigvecseries[:,itrial,:NE]),axis=1),(ngavg,1))
    meancol=np.repeat(meancol,NE,axis=1)
    overlapvector[:,0,1,:]=np.squeeze(Reigvecseries[:,itrial,:NE])-meancol
    meancol=np.reshape(np.mean(np.squeeze(Reigvecseries[:,itrial,NE:]),axis=1),(ngavg,1))
    meancol=np.repeat(meancol,NE,axis=1)
    overlapvector[:,1,1,:]=np.squeeze(Reigvecseries[:,itrial,NE:])-meancol


    for i in range(ngavg):
        vectorEmat,vectorImat=np.squeeze(overlapvector[i,0,:,:]),np.squeeze(overlapvector[i,1,:,:])
        perturbEmat,perturbImat=np.squeeze(overlapperturb[i,0,:,:]),np.squeeze(overlapperturb[i,1,:,:])
        vcovE,vcovI=vectorEmat@vectorEmat.T,vectorImat@vectorImat.T
        pcovE,pcovI=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T

        vvE,dirvE=la.eig(vcovE)
        vvI,dirvI=la.eig(vcovI)
        vpE,dirpE=la.eig(pcovE)
        vpI,dirpI=la.eig(pcovI)
        directionvecE[i,itrial,:,:],directionvecI[i,:,:]=dirvE,dirvI
        directionperturbE[i,itrial,:,:],directionperturbI[i,:,:]=dirpE,dirpI
        ampvecE[i,itrial,:],ampvecI[i,itrial,:]=vvE*dh,vvI
        ampperturbE[i,itrial,:],ampperturbI[i,itrial,:]=vpE*dh,vpI*dh


## averaging
directionvecEavg,directionvecIavg,directionperturbEavg,directionperturbIavg=np.mean(directionvecE,axis=1),np.mean(directionvecI,axis=1),np.mean(directionperturbE,axis=1),np.mean(directionperturbI,axis=1)
ampvecEavg,ampvecIavg,ampperturbEavg,ampperturbIavg=np.mean(ampvecE,axis=1),np.mean(ampvecI,axis=1),np.mean(ampperturbE,axis=1),np.mean(ampperturbI,axis=1)



<ipython-input-243-9f8990a19c96>:46: RuntimeWarning: divide by zero encountered in true_divide
  coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
<ipython-input-243-9f8990a19c96>:88: ComplexWarning: Casting complex values to real discards the imaginary part
  directionvecE[i,itrial,:,:],directionvecI[i,:,:]=dirvE,dirvI
<ipython-input-243-9f8990a19c96>:89: ComplexWarning: Casting complex values to real discards the imaginary part
  directionperturbE[i,itrial,:,:],directionperturbI[i,:,:]=dirpE,dirpI
<ipython-input-243-9f8990a19c96>:90: ComplexWarning: Casting complex values to real discards the imaginary part
  ampvecE[i,itrial,:],ampvecI[i,itrial,:]=vvE*dh,vvI
<ipython-input-243-9f8990a19c96>:91: ComplexWarning: Casting complex values to real discards the imaginary part
  ampperturbE[i,itrial,:],ampperturbI[i,itrial,:]=vpE*dh,vpI*dh


In [246]:
# ## averaging
# directionvecEavg,directionvecIavg,directionperturbEavg,directionperturbIavg=np.mean(directionvecE,axis=1),np.mean(directionvecI,axis=1),np.mean(directionperturbE,axis=1),np.mean(directionperturbI,axis=1)
# ampvecEavg,ampvecIavg,ampperturbEavg,ampperturbIavg=np.mean(ampvecE,axis=1),np.mean(ampvecI,axis=1),np.mean(ampperturbE,axis=1),np.mean(ampperturbI,axis=1)

fig,ax=plt.subplots(figsize=(5,5)) 
for idxeta in np.arange(1,8):
    heta=gaverageseries[-1]*0.1
    # ax.plot([0,0+ampvecEavg[idxeta,0]*directionvecEavg[idxeta,0,0]],[0,0+ampvecEavg[idxeta,0]*directionvecEavg[idxeta,1,0]],color='r',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecIavg[idxeta,0]*directionvecIavg[idxeta,0,0]],[0,0+ampvecIavg[idxeta,0]*directionvecIavg[idxeta,1,0]],color='orange',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,0,0]],[0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,1,0]],color='r',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,0,0]],[0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,1,0]],color='orange',linestyle='-',linewidth=1.5)

    vecEx,vecEy=ampvecEavg[idxeta,0]*directionvecEavg[idxeta,0,0],ampvecEavg[idxeta,0]*directionvecEavg[idxeta,1,0]
    vecIx,vecIy=ampvecIavg[idxeta,0]*directionvecIavg[idxeta,0,0],ampvecIavg[idxeta,0]*directionvecIavg[idxeta,1,0]
    if idxeta<7:
        ax.plot([vecEx,vecEx+ampperturbEavg[idxeta,0]*directionperturbEavg[idxeta,0,0]*heta],[vecEy,vecEy+ampperturbEavg[idxeta,0]*directionperturbEavg[idxeta,1,0]*heta],color='b',linestyle='--',linewidth=1.5)
        ax.plot([vecIx,vecIx+ampperturbIavg[idxeta,0]*directionperturbIavg[idxeta,0,0]*heta],[vecIy,vecIy+ampperturbIavg[idxeta,0]*directionperturbIavg[idxeta,1,0]*heta],color='g',linestyle='--',linewidth=1.5)
    # ax.plot([vecEx,vecEx+ampperturbE[idxeta,1]*directionperturbE[idxeta,0,0]*heta],[vecEy,vecEy+ampperturbE[idxeta,1]*directionperturbE[idxeta,1,0]*heta],color='b',linestyle='--',linewidth=1.5)
    # ax.plot([vecIx,vecIx+ampperturbI[idxeta,1]*directionperturbI[idxeta,0,0]*heta],[vecIy,vecIy+ampperturbI[idxeta,1]*directionperturbI[idxeta,1,0]*heta],color='g',linestyle='--',linewidth=1.5)

xticks = np.linspace(-0.05,0.05,2)
xlims = [-0.05,0.05]
yticks = np.linspace(-0.05,0.05,2)
ylims = [-0.05,0.05]
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')

In [103]:
'''
loading patterns
'''
# xticks = np.linspace(-0.15,0.10,2)
# xlims = [-0.15,0.10]
# yticks = np.linspace(-0.05,0.2,2)
# ylims = [-0.05,0.2]

## withoueigenvalue
xticks = np.linspace(-0.1,0.1,2)
xlims = [-0.1,0.1]
yticks = np.linspace(-0.02,0.08,2)
ylims = [-0.02,0.08]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''
# pearsoncorr_rho=np.zeros((3,ngavg,neta,ntrial,2,nrank))
# thetamaxseries=np.zeros((3,ngavg,neta,ntrial,2,2))
# eigtotalseries=np.zeros((3,ngavg,neta,ntrial,nrank))
idxtrial=5#16#
idxgavg=4#9# 
cm='mc'
# fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
# ## directly use
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0])*Beigvseries[idxgavg,idxtrial,0],np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])*Beigvseries[idxgavg,idxtrial,0]
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
# ## directly use without eigvalue
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
## reload from recording
mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE]),np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:]),np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:])
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

## keep the same sign
if meanmE[0]<0:
    meanmE,meanmI,meannE,meannI=-meanmE,-meanmI,-meannE,-meannI
    mEvec,mIvec,nEvec,nIvec=-mEvec,-mIvec,-nEvec,-nIvec
''' calculate directions '''
noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec,mIvec,s=10.0,c=cm[1],alpha=0.25)
ax.scatter(nEvec,mEvec,s=10.0,c=cm[0],alpha=0.25)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=cm[0],linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=cm[0],linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=cm[1],linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=cm[1],linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=cm[0])
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=cm[1])
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')
print(gaverageseries[idxgavg])

            


750
(2, 2)
(2, 2)
0.5
<ipython-input-103-321248a7bc9d>:42: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec)
<ipython-input-103-321248a7bc9d>:49: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-103-321248a7bc9d>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-103-321248a7bc9d>:64: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-103-321248a7bc9d>:75: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
<ipython-input-103-321248a7bc9d>:76: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]


# with symmetry
## loading data

In [261]:
strfile='2021-03-27-22_40_39_perturbationHomoeta_data' # 750
# strfile='2021-03-27-23_15_34_perturbationHomoeta_data' # 500
# strfile='2021-03-28-16_34_16_perturbationHomoeta_data' #1000

strfile='2021-03-28-10_09_24_perturbationHomoeta_data' # 750
# strfile='2021-03-28-10_14_57_perturbationHomoeta_data' # 500
# strfile='2021-03-28-16_15_47_perturbationHomoeta_data' # 1000
data = np.load(strfile+'.npz')
etaseries=data['etaseries']
ngavg=len(gaverageseries)
eigvAm=data['eigvAm']
eigvalseries=data['eigvalseries']
statesReig=data['statesReig']
statesLeig=data['statesLeig']
meigvecAm=data['meigvecAm']
neigvecAm=data['neigvecAm']
gYbasexee=data['gYbasexee']
Jparameters=data['Jparameters']
# right and left eigenvectors
Reigvecseries,Leigvecseries,Beigvseries=data['Reigvecseries'],data['Leigvecseries'],data['Beigvseries']
ntrial=np.shape(Reigvecseries)[1]
x0,y0=data['x0'],data['y0']
xAm,yAm=data['xAm'],data['yAm']
Randgain=data['Randgain']
xee,xei,xie,xii=Randgain[0],Randgain[1],Randgain[2],Randgain[3]
coeffeta=data['coeffeta']

print('scale:',Jparameters[4])
NE=int(Jparameters[4])
N = NE*2
print('JEE:',Jparameters[0])
etaset=data['etaset']
signeta=data['signeta']
gaverage=data['gaverage']
JE,JI=Jparameters[0],Jparameters[1]
print('eta:',etaset,'; signs:',signeta)

neta=len(etaseries)

scale: 750.0
JEE: 1.0
eta: [1. 1. 1.] ; signs: [1. 1. 1.]


In [227]:
'''
determinant lemma, odd terms 
even order terms should be maitained
combination
'''
from scipy.special import comb, perm
gaverage
xticks = np.linspace(-2.91,-2.5,2)
xlims = [-2.91,-2.5]
yticks = np.linspace(-2.91,-2.5,2)
ylims = [-2.91,-2.5]

coeff=coeffeta.copy()
nAm=np.reshape(yAm*eigvAm[0],(NE*2,1))
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
y0E,y0I=np.mean(y0[:NE,0]),np.mean(y0[NE:,0])
x0E,x0I=np.mean(x0[:NE,0]),np.mean(x0[NE:,0])
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:]),axis=1)
cutoff = 2

for idxeta,eta in enumerate(etaseries):
    etaE,etaB,etaI=eta*coeff[0],eta*coeff[1],eta*coeff[2]
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXT=np.eye(NE*2)
    bXXT[:NE,:NE]*=(etaE*signeta[0]+etaB*signeta[1])/4
    bXXT[NE:,NE:]*=(etaI*signeta[2]+etaB*signeta[1])/4
    # cut off = 2
    C=comb(2,1)
    print(C)
    B2=C*(nAm.T@la.matrix_power(bXXT,1)@mAm)*gaverage**2
    coeff2 = [1,-eigvAm[0],0,-B2]
    C=comb(4,2)
    B4=C*(nAm.T@la.matrix_power(bXXT,2)@mAm)*gaverage**4
    # B4=(nAm.T@(Y_t@YT_t@Y_t@YT_t)@mAm)*fory**2*foryt**2*6
    coeff4 = [1,-eigvAm[0],0,-B2,0,-B4]
    lambda_test2=np.roots(coeff2)
    lambda_theo2[idxeta]=lambda_test2[0]
    lambda_test4=np.roots(coeff4)
    lambda_theo4[idxeta]=lambda_test4[0]
    print(lambda_test2[0],lambda_test4[0])
fig,ax=plt.subplots(figsize=(5,5))  
ax.plot(lambda_num,lambda_theo2,'b',label='2 cut off')
ax.plot(lambda_num,lambda_theo4,'r',label='4 cut off')
# ax.plot(lambda_num,lambda_theo,'b')
ax.plot(lambda_num,lambda_num,'k',label='real')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')
ax.legend()

# fig,ax=plt.subplots(figsize=(5,3))  
# ax.plot(etaseries,lambda_num,'b',label='2 cut off')
# ax.set_aspect('equal')
# # ax.scatter(np.real(Beigvseries[neta-5,8,1:]),np.imag(Beigvseries[neta-5,8,1:]))
# # ax.set_aspect('equal')


2.0
-2.499999999999994 -2.499999999999994
2.0
(-2.501775255656491+0j) (-2.5017771438905996+0j)
2.0
(-2.5035454920136706+0j) (-2.50355301292924+0j)
2.0
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/lib/polynomial.py:238: ComplexWarning: Casting complex values to real discards the imaginary part
  p = p.astype(float)
<ipython-input-227-84306781a13d>:42: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_theo2[idxeta]=lambda_test2[0]
<ipython-input-227-84306781a13d>:44: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_theo4[idxeta]=lambda_test4[0]
(-2.505310746148103+0j) (-2.5053275966690336+0j)
2.0
(-2.5070710546913197+0j) (-2.50710088488641+0j)
2.0
(-2.5088264538372123+0j) (-2.5088728675777956+0j)
2.0
(-2.510576979349249+0j) (-2.51064353495629+0j)
2.0
(-2.5123226665675853+0j) (-2.5124128774483934+0j)
2.0
(-2.514063550415969+0j) (-2.51418088569081+0j)
2.0
(-2.515799665408548+0j) (-2.515947550527221+0

In [228]:
'''
determinant lemma, odd terms 
even order terms should be maitained
original Algorithm, miss terms
'''
xticks = np.linspace(-2.91,-2.5,2)
xlims = [-2.91,-2.5]
yticks = np.linspace(-2.91,-2.5,2)
ylims = [-2.91,-2.5]

coeff=coeffeta.copy()
y0E,y0I=np.mean(y0[:NE,0]),np.mean(y0[NE:,0])
x0E,x0I=np.mean(x0[:NE,0]),np.mean(x0[NE:,0])
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:]),axis=1)
cutoff = 2
for idxeta,eta in enumerate(etaseries):
    etaf,etam,etal=eta*coeff[0],coeff[1]*eta,eta*coeff[2]
    bcoeffE0,bcoeffI0=(signeta[0]*etaf+signeta[1]*etam)*1/2.0,(signeta[2]*etal+signeta[1]*etam)*1/2.0
    print(bcoeffI0,bcoeffE0)
    B2=(bcoeffE0*JE-bcoeffI0*JI)*gaverage**2
    coeff2 = [1,-eigvAm[0],0,-B2]
    B4=(bcoeffE0**2*JE-bcoeffI0**2*JI)*gaverage**4
    coeff4 = [1,-eigvAm[0],0,-B2,0,-B4]
    # B2=(bcoeffE0*y0E*x0E*NE)+(bcoeffI0*y0I*x0I*NE)
    # coeff2 = [1,-eigvJ0[0],0,-B2*eigvJ0[0]]
    # B4=(bcoeffE0**2*y0E*x0E*NE)+(bcoeffI0**2*y0I*x0I*NE)
    # coeff4 = [1,-eigvJ0[0],0,-B2*eigvJ0[0],0,-B4*eigvJ0[0]]#(eta*eigvAm[0]*gaverage])
    lambda_test2=np.roots(coeff2)
    lambda_theo2[idxeta]=lambda_test2[0]
    lambda_test4=np.roots(coeff4)
    lambda_theo4[idxeta]=lambda_test4[0]
    print(lambda_test2[0],lambda_test4[0])
fig,ax=plt.subplots(figsize=(5,5))  
ax.plot(lambda_num,lambda_theo2,'b',label='2 cut off')
ax.plot(lambda_num,lambda_theo4,'r',label='4 cut off')
# ax.plot(lambda_num,lambda_theo,'b')
ax.plot(lambda_num,lambda_num,'k',label='real')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')
ax.legend()

fig,ax=plt.subplots(figsize=(5,3))  
ax.plot(etaseries,lambda_num,'b',label='2 cut off')
ax.set_aspect('equal')
# ax.scatter(np.real(Beigvseries[neta-5,8,1:]),np.imag(Beigvseries[neta-5,8,1:]))
# ax.set_aspect('equal')


0.0 0.0
(-2.499999999999994+0j) (-2.499999999999994+0j)
0.1111111111111111 0.1111111111111111
(-2.50177525565649-4.163336342344337e-17j) (-2.5017765144804978-1.3877787807814457e-17j)
0.2222222222222222 0.2222222222222222
(-2.503545492013669-2.7755575615628914e-17j) (-2.50355050597744-5.551115123125783e-17j)
0.3333333333333333 0.3333333333333333
(-2.505310746148104+4.163336342344337e-17j) (-2.505321979929298-9.71445146547012e-17j)
0.4444444444444444 0.4444444444444444
(-2.507071054691319+8.326672684688674e-17j) (-2.5070909418028657-9.71445146547012e-17j)
0.5555555555555556 0.5555555555555556
(-2.508826453837214-2.7755575615628914e-17j) (-2.508857397092144-1.1102230246251565e-16j)
0.6666666666666666 0.6666666666666666
(-2.5105769793492545-1.1102230246251565e-16j) (-2.510621351317263-5.551115123125783e-17j)
0.7777777777777777 0.7777777777777777
(-2.5123226665675897+5.551115123125783e-17j) (-2.5123828100233334+2.498001805406602e-16j)
0.8888888888888888 0.8888888888888888
(-2.51406355041597

In [262]:
'''
# <l',r'> reflects the direction of changing
'''

'''
variance of r and variance of l <r.Tr>, <l.T,l>
overlap between r and l <l.Tr>
'''
xticks = np.linspace(-0.01,0.1,2)
xlims = [-0.01,0.1]
yticks = np.linspace(-0.01,0.1,2)
ylims = [-0.01,0.1]
## reconstruct Am
JEE,JEI,JIE,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
NI=NE
Nparams=np.array([NE,NI])
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
vcovEmat,vcovImat=np.zeros((neta,ntrial,2,2)),np.zeros((neta,ntrial,2,2))
pcovEmat,pcovImat=np.zeros((neta,ntrial,2,2)),np.zeros((neta,ntrial,2,2))
directionvecE,directionvecI,directionperturbE,directionperturbI=np.zeros((neta,2,2)),np.zeros((neta,2,2)),np.zeros((neta,2,2)),np.zeros((neta,2,2))
ampvecE,ampvecI,ampperturbE,ampperturbI=np.zeros((neta,2)),np.zeros((neta,2)),np.zeros((neta,2)),np.zeros((neta,2))
ntrial = np.shape(Leigvecseries)[1]
for itrial in range(ntrial):
    ## choose one trial and the random matrix
    # itrial=6
    gY=np.squeeze(gYbasexee[itrial,:,:])
    ## reconstruct the non-symmetric matrix (original random matrix with \eta=0)
    X0=gY.copy()
    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,eigrvecJ0=la.eig(J0)
    inveigrvecJ0=la.inv(eigrvecJ0)
    meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    neig0=neig0.copy().T
    print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
    for i in range(nrank):
        neig0[:,i]*=eigvJ0[i]
    for j in range(1,N):
        neig0[:,j]*=eigvJ0[j]

    leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    reig0=meig0.copy()
    normval=np.sum(reig0*neig0.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig0=neig0.copy()/normval.copy()
    ''' calculate s0'''
    Z = np.zeros((NE*2-1,NE*2-1))
    for i in range(NE*2-1):
        b1=eigvJ0[i+1]
        Z[i,i]=1/(b1-eigvJ0[0])
    S0=reig0[:,1:]@Z@leig0[:,1:].T
    print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
    print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
    print('eigenvalues:',eigvJ0[:nrank])
    '''## for reference '''
    x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    ''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
    tildex,tildey=np.reshape(x0[:,0].copy(),(NE*2,1)),np.reshape(y0[:,0].copy(),(NE*2,1))
    ## make sure the E in x is positive, if negative then all x and y should change sign
    if (np.mean(tildex[:NE])*xAm[0,0])<0:
        tildex*=(-1)
        tildey*=(-1)
    x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
    x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))

    '''
    rho_prime=eta/(2*sqrt(1-eta**2))
    rho'/2/sqrt(rho) to 2nd=(1+np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
    rho'/2/sqrt(1-rho) to 2nd=(1-np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
    '''
    dh=etaseries[1]-etaseries[0]
    rhoseries=(1-np.sqrt(1-etaseries**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
    sqrt_1meta2= np.sqrt(1-etaseries**2)
    coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
    overlapperturb,overlapvector = np.zeros((neta,2,2,NE),dtype=complex),np.zeros((neta,2,2,NE),dtype=complex)
    # EI,LR,:
    for i in range(neta-1):
        Jprimemat=np.sqrt(coeffsqrtYT[i])*gY.copy().T-np.sqrt(coeffsqrtY[i])*gY
        leftperturb=np.squeeze(-np.squeeze(y0.T@Jprimemat))/(-eigvAm[0])# left perturbation E
        rightperturb=np.squeeze(-np.squeeze(Jprimemat@x0))/(-eigvAm[0])#*Beigvseries[i,itrial,0]# right perturbation E
        overlapperturb[i,0,0,:],overlapperturb[i,0,1,:]=leftperturb[:NE],rightperturb[:NE]
        overlapperturb[i,1,0,:],overlapperturb[i,1,1,:]=leftperturb[NE:],rightperturb[NE:]
        overlapvector[i,0,0,:]=np.squeeze(Leigvecseries[i,itrial,:NE])-np.mean(np.squeeze(Leigvecseries[i,itrial,:NE]))
        overlapvector[i,1,0,:]=np.squeeze(Leigvecseries[i,itrial,NE:])-np.mean(np.squeeze(Leigvecseries[i,itrial,NE:]))
        overlapvector[i,0,1,:]=np.squeeze(Reigvecseries[i,itrial,:NE])-np.mean(np.squeeze(Reigvecseries[i,itrial,:NE]))
        overlapvector[i,1,1,:]=np.squeeze(Reigvecseries[i,itrial,NE:])-np.mean(np.squeeze(Reigvecseries[i,itrial,NE:]))
    ## E population 
    # vectorcovE,perturbationcovE = np.zeros((neta,2,2)),np.zeros((neta,2,2))

    for i in range(neta):
        vectorEmat,vectorImat=np.squeeze(overlapvector[i,0,:,:]),np.squeeze(overlapvector[i,1,:,:])
        perturbEmat,perturbImat=np.squeeze(overlapperturb[i,0,:,:]),np.squeeze(overlapperturb[i,1,:,:])
        vcovEmat[i,itrial,:,:],vcovImat[i,itrial,:,:]=vectorEmat@vectorEmat.T,vectorImat@vectorImat.T
        pcovEmat[i,itrial,:,:],pcovImat[i,itrial,:,:]=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T

## averaging
vcovEmatavg,vcovImatavg=np.mean(vcovEmat,axis=1),np.mean(vcovImat,axis=1)
pcovEmatavg,pcovImatavg=np.mean(pcovEmat,axis=1),np.mean(pcovImat,axis=1)
for i in range(neta):
    vvE,dirvE=la.eig(np.squeeze(vcovEmatavg[i,:,:]))
    vvI,dirvI=la.eig(np.squeeze(vcovImatavg[i,:,:]))
    vpE,dirpE=la.eig(np.squeeze(pcovEmatavg[i,:,:]))
    vpI,dirpI=la.eig(np.squeeze(pcovImatavg[i,:,:]))
    directionvecE[i,:,:],directionvecI[i,:,:]=dirvE,dirvI
    directionperturbE[i,:,:],directionperturbI[i,:,:]=dirpE,dirpI
    ampvecE[i,:],ampvecI[i,:]=vvE*dh,vvI*dh
    ampperturbE[i,:],ampperturbI[i,:]=vpE*dh,vpI*dh



test normal: (1.000000000000091+4.891068383059579e-14j)
test normal 2: (0.999999999999999+1.3883951931889808e-28j)
test normal 3: (-2.4778946397803274-1.211955212918126e-13j)
eigenvalues: [-2.47789464+0.j]
<ipython-input-262-f843d5affd88>:53: ComplexWarning: Casting complex values to real discards the imaginary part
  Z[i,i]=1/(b1-eigvJ0[0])
<ipython-input-262-f843d5affd88>:79: RuntimeWarning: divide by zero encountered in true_divide
  coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
<ipython-input-262-f843d5affd88>:98: ComplexWarning: Casting complex values to real discards the imaginary part
  vcovEmat[i,itrial,:,:],vcovImat[i,itrial,:,:]=vectorEmat@vectorEmat.T,vectorImat@vectorImat.T
<ipython-input-262-f843d5affd88>:99: ComplexWarning: Casting complex values to real discards the imaginary part
  pcovEmat[i,itrial,:,:],pcovImat[i,itrial,:,:]=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T
test normal: (0.9999999999999536+1.697956992002

In [263]:
# ## averaging
# directionvecEavg,directionvecIavg,directionperturbEavg,directionperturbIavg=np.mean(directionvecE,axis=1),np.mean(directionvecI,axis=1),np.mean(directionperturbE,axis=1),np.mean(directionperturbI,axis=1)
# ampvecEavg,ampvecIavg,ampperturbEavg,ampperturbIavg=np.mean(ampvecE,axis=1),np.mean(ampvecI,axis=1),np.mean(ampperturbE,axis=1),np.mean(ampperturbI,axis=1)

fig,ax=plt.subplots(figsize=(5,5)) 
for idxeta in np.arange(1,8):
    heta=0.3
    ax.plot([0,0+ampvecE[idxeta,0]*directionvecE[idxeta,0,0]],[0,0+ampvecE[idxeta,0]*directionvecE[idxeta,1,0]],color='r',linestyle='-',linewidth=1.5)
    ax.plot([0,0+ampvecI[idxeta,0]*directionvecI[idxeta,0,0]],[0,0+ampvecI[idxeta,0]*directionvecI[idxeta,1,0]],color='orange',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,0,0]],[0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,1,0]],color='r',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,0,0]],[0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,1,0]],color='orange',linestyle='-',linewidth=1.5)

    vecEx,vecEy=ampvecE[idxeta,0]*directionvecE[idxeta,0,0],ampvecE[idxeta,0]*directionvecE[idxeta,1,0]
    vecIx,vecIy=ampvecI[idxeta,0]*directionvecI[idxeta,0,0],ampvecI[idxeta,0]*directionvecI[idxeta,1,0]
    if idxeta<7:
        ax.plot([vecEx,vecEx+ampperturbE[idxeta,1]*directionperturbE[idxeta,0,1]*heta],[vecEy,vecEy+ampperturbE[idxeta,1]*directionperturbE[idxeta,1,1]*heta],color='b',linestyle='--',linewidth=1.5)
        ax.plot([vecIx,vecIx+ampperturbI[idxeta,1]*directionperturbI[idxeta,0,1]*heta],[vecIy,vecIy+ampperturbI[idxeta,1]*directionperturbI[idxeta,1,1]*heta],color='g',linestyle='--',linewidth=1.5)
    # ax.plot([vecEx,vecEx+ampperturbE[idxeta,1]*directionperturbE[idxeta,0,0]*heta],[vecEy,vecEy+ampperturbE[idxeta,1]*directionperturbE[idxeta,1,0]*heta],color='b',linestyle='--',linewidth=1.5)
    # ax.plot([vecIx,vecIx+ampperturbI[idxeta,1]*directionperturbI[idxeta,0,0]*heta],[vecIy,vecIy+ampperturbI[idxeta,1]*directionperturbI[idxeta,1,0]*heta],color='g',linestyle='--',linewidth=1.5)

# xticks = np.linspace(-0.0,0.15,2)
# xlims = [-0.00,0.15]
# yticks = np.linspace(-0.0,0.035,2)
# ylims = [-0.0,0.025]
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')

In [252]:
vecEx

0.0

In [210]:
'''
loading patterns with symmetry
'''

## withoueigenvalue
xticks = np.linspace(-0.1,0.1,2)
xlims = [-0.1,0.1]
yticks = np.linspace(-0.02,0.08,2)
ylims = [-0.02,0.08]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''
idxtrial=3#16#
idxgavg=1#9# 
cm='mc'
fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
# ## directly use
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0])*Beigvseries[idxgavg,idxtrial,0],np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])*Beigvseries[idxgavg,idxtrial,0]
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
# ## directly use without eigvalue
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
## reload from recording
mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE]),np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:]),np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:])
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec,mIvec,s=10.0,c=cm[1],alpha=0.25)
ax.scatter(nEvec,mEvec,s=10.0,c=cm[0],alpha=0.25)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=cm[0],linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=cm[0],linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=cm[1],linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=cm[1],linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=cm[0])
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=cm[1])
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
ax.set_aspect('equal')
print(gaverageseries[idxgavg])

            


500
(2, 2)
(2, 2)
0.2
<ipython-input-210-ac8a17544e29>:35: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec)
<ipython-input-210-ac8a17544e29>:42: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-210-ac8a17544e29>:50: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-210-ac8a17544e29>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-210-ac8a17544e29>:64: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
<ipython-input-210-ac8a17544e29>:65: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
